# Create migration matrix
### Load libraries and datasets

In [39]:
import pandas as pd
import numpy as np
from scipy import linalg

effective_distances = pd.read_csv('./original_data/effective.distance.matrix.country.csv', header=0)
population_sizes = pd.read_csv('./output/census_2013.csv', header=0)
effective_distances.drop(effective_distances.columns[0], axis=1, inplace=True)
alpha2_codes = effective_distances.columns
effective_distances.shape

(228, 228)

## Flux matrix
Invert effective distance function to get flux. Off diagonals matter only, since diagonal is zero in effective distance matrix.

In [40]:
flux_matrix = effective_distances.apply(lambda x: np.exp(1 - x))

## Clean flux matrix
Remove rows and columns of countries in flux matrix that aren't in population dataset.

In [41]:
missing_countries_columns = []
missing_countries_rows = []
for index, alpha2 in enumerate(alpha2_codes):
    country_mask = population_sizes['alpha2'] == alpha2
    country_data = population_sizes[country_mask]
    if len(country_data['population']) == 0:
        missing_countries_columns.append(alpha2)
        missing_countries_rows.append(index)
flux_matrix = flux_matrix.drop(missing_countries_columns, axis=1).drop(flux_matrix.index[missing_countries_rows])
np.fill_diagonal(flux_matrix.values, 0)
flux_matrix.index = flux_matrix.columns

In [42]:
flux_matrix

,AE,AF,AG,AL,AM,AO,AR,AS,AT,AU,...,VE,VG,VI,VN,VU,WS,YE,ZA,ZM,ZW
AE,0.000000,0.009912,0.000027,0.000103,0.001148,0.001801,0.000294,2.634900e-06,0.004773,0.029821,...,0.000069,3.484592e-06,0.000057,0.002402,6.409661e-05,1.867199e-05,0.004359,0.014089,0.001956,0.000351
AF,0.578062,0.000000,0.000006,0.000104,0.000244,0.000383,0.000063,5.603303e-07,0.012658,0.006342,...,0.000015,7.410236e-07,0.000012,0.000511,1.363061e-05,3.970733e-06,0.000927,0.002996,0.000416,0.000075
AG,0.002067,0.000008,0.000000,0.000039,0.000005,0.000013,0.000502,1.694864e-05,0.000436,0.000953,...,0.000446,8.488147e-04,0.001611,0.000026,2.048515e-06,4.834559e-06,0.000003,0.000424,0.000014,0.000011
AL,0.004458,0.000077,0.000021,0.000000,0.000087,0.000022,0.000735,7.057790e-07,0.076812,0.000049,...,0.000340,9.333760e-07,0.000015,0.000034,1.051199e-07,2.013217e-07,0.000032,0.000121,0.000004,0.000004
AM,0.055473,0.000202,0.000003,0.000096,0.000000,0.000037,0.000024,5.841516e-07,0.041402,0.000609,...,0.000015,7.725267e-07,0.000013,0.001946,1.308039e-06,3.810450e-07,0.000089,0.000288,0.000040,0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WS,0.004768,0.000017,0.000015,0.000001,0.000002,0.000005,0.000199,1.828579e-01,0.000025,0.158273,...,0.000093,4.665183e-06,0.000076,0.000635,8.778361e-04,0.000000e+00,0.000008,0.000626,0.000010,0.000016
YE,0.223835,0.000816,0.000002,0.000036,0.000095,0.000222,0.000024,2.169690e-07,0.000475,0.002456,...,0.000006,2.869363e-07,0.000005,0.000198,5.277991e-06,1.537532e-06,0.000000,0.001160,0.000211,0.000264
ZA,0.106421,0.000388,0.000041,0.000021,0.000045,0.023831,0.005107,2.340861e-06,0.000959,0.029584,...,0.000064,3.095733e-06,0.000050,0.000166,6.358722e-05,1.852360e-05,0.000171,0.000000,0.044434,0.067659
ZM,0.109331,0.000399,0.000010,0.000008,0.000046,0.005939,0.000617,2.830141e-07,0.000192,0.003577,...,0.000008,4.631309e-07,0.000006,0.000097,7.687806e-06,2.239536e-06,0.000354,0.328645,0.000000,0.223755


In [43]:
population_sizes = population_sizes[population_sizes['alpha2'].isin(alpha2_codes)]
population_sizes.set_index('alpha2', inplace=True)
population_sizes

,population
alpha2,
AF,32716210
AL,2889104
DZ,38760168
AS,52217
AO,27128337
...,...
VN,91235504
VI,107882
YE,27753304


## Calculate transition matrix 
Using assumption that flux of a location is proportional to it's size, the expression for the transition matrix can be derived from the supplementary text of https://www.science.org/doi/10.1126/science.1245200#supplementary-materials

Specifically the transition matrix entries cancels down to $$w_{nm} = \frac{P_{nm} \Phi}{\Omega}$$ where $P_{nm}$ is the proportional flux matrix (calculated in this code as `flux_matrix`). 

The diagonals can then be calculated using the fact that rows will sum to $0$ in a transition matrix.



In [44]:
Phi = 8.91 * 10 ** 6
Omega = sum(population_sizes['population'])
transition_matrix = flux_matrix * Phi / Omega

Now just need to set diagonals to make sure rows sum to zero

In [45]:
total_rates = -1 * transition_matrix.sum(axis=1)
np.fill_diagonal(transition_matrix.values, total_rates)


In [46]:
transition_matrix

,AE,AF,AG,AL,AM,AO,AR,AS,AT,AU,...,VE,VG,VI,VN,VU,WS,YE,ZA,ZM,ZW
AE,-0.001236,1.212217e-05,3.262928e-08,1.262616e-07,1.404304e-06,2.202253e-06,3.600206e-07,3.222512e-09,5.837302e-06,3.647108e-05,...,8.488996e-08,4.261696e-09,6.923754e-08,2.937927e-06,7.839088e-08,2.283606e-08,5.330503e-06,1.723073e-05,2.392701e-06,4.288790e-07
AF,0.000707,-1.409465e-03,6.938852e-09,1.269141e-07,2.986354e-07,4.683249e-07,7.656097e-08,6.852903e-10,1.548137e-05,7.755838e-06,...,1.805246e-08,9.062802e-10,1.472386e-08,6.247713e-07,1.667038e-08,4.856252e-09,1.133570e-06,3.664239e-06,5.088251e-07,9.120420e-08
AG,0.000003,9.216996e-09,-1.269019e-03,4.830520e-08,6.039971e-09,1.550197e-08,6.135663e-07,2.072838e-08,5.335222e-07,1.165609e-06,...,5.460434e-07,1.038110e-06,1.970739e-06,3.189549e-08,2.505357e-09,5.912720e-09,4.053006e-09,5.185250e-07,1.665809e-08,1.290627e-08
AL,0.000005,9.413520e-08,2.594960e-08,-1.415662e-03,1.064179e-07,2.736366e-08,8.985126e-07,8.631757e-10,9.394244e-05,5.981338e-08,...,4.158046e-07,1.141529e-09,1.854583e-08,4.128122e-08,1.285628e-10,2.462187e-10,3.967856e-08,1.483921e-07,4.767231e-09,4.860393e-09
AM,0.000068,2.473808e-07,3.567516e-09,1.177834e-07,-1.450828e-03,4.494204e-08,2.982451e-08,7.144241e-10,5.063537e-05,7.442764e-07,...,1.881992e-08,9.448089e-10,1.534982e-08,2.379472e-06,1.599746e-09,4.660223e-10,1.087812e-07,3.516328e-07,4.882857e-08,8.752263e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WS,0.000006,2.126146e-08,1.895031e-08,1.334410e-09,2.463054e-09,6.708106e-09,2.434665e-07,2.236373e-04,3.023281e-08,1.935699e-04,...,1.136509e-07,5.705571e-09,9.269544e-08,7.767753e-07,1.073603e-06,-1.307646e-03,9.349339e-09,7.651647e-07,1.250764e-08,1.904522e-08
YE,0.000274,9.981917e-07,2.686836e-09,4.418785e-08,1.156364e-07,2.716619e-07,2.964565e-08,2.653555e-10,5.806200e-07,3.003186e-06,...,6.990205e-09,3.509264e-10,5.701317e-09,2.419216e-07,6.455043e-09,1.880419e-09,-1.305220e-03,1.418853e-06,2.576099e-07,3.223572e-07
ZA,0.000130,4.745835e-07,4.988158e-08,2.575463e-08,5.497857e-08,2.914539e-05,6.246307e-06,2.862899e-09,1.173237e-06,3.618124e-05,...,7.840271e-08,3.786116e-09,6.151105e-08,2.033665e-07,7.776789e-08,2.265458e-08,2.086894e-07,-1.210775e-03,5.434323e-05,8.274771e-05
ZM,0.000134,4.875614e-07,1.179937e-08,1.027607e-08,5.648201e-08,7.262904e-06,7.551893e-07,3.461295e-10,2.347800e-07,4.374375e-06,...,9.479024e-09,5.664143e-10,7.436793e-09,1.181653e-07,9.402274e-09,2.738978e-09,4.326563e-07,4.019365e-04,-1.332462e-03,2.736555e-04


### Convert to a probability matrix

In [47]:
probability_matrix = pd.DataFrame(linalg.expm(transition_matrix))
probability_matrix

,0,1,2,3,4,5,6,7,8,9,...,199,200,201,202,203,204,205,206,207,208
0,0.998765,1.210741e-05,3.269566e-08,1.264732e-07,1.403080e-06,2.200062e-06,3.607328e-07,3.232733e-09,5.834959e-06,3.643280e-05,...,8.513671e-08,4.275702e-09,6.932091e-08,2.937418e-06,7.844597e-08,2.288600e-08,5.325797e-06,1.721450e-05,2.390485e-06,4.296197e-07
1,0.000706,9.985915e-01,6.970242e-09,1.271387e-07,2.993908e-07,4.686308e-07,7.685915e-08,6.889687e-10,1.546595e-05,7.760658e-06,...,1.814737e-08,9.112394e-10,1.477372e-08,6.259062e-07,1.670999e-08,4.875059e-09,1.134642e-06,3.667199e-06,5.091950e-07,9.152200e-08
2,0.000003,9.259144e-09,9.987318e-01,4.844886e-08,6.090581e-09,1.558551e-08,6.149357e-07,2.075105e-08,5.349621e-07,1.167407e-06,...,5.488585e-07,1.040009e-06,1.972406e-06,3.216713e-08,2.515058e-09,5.933194e-09,4.083048e-09,5.193074e-07,1.669675e-08,1.295255e-08
3,0.000005,9.430029e-08,2.601171e-08,9.985853e-01,1.068826e-07,2.751115e-08,8.993490e-07,8.660300e-10,9.383238e-05,6.025537e-08,...,4.161870e-07,1.148791e-09,1.861255e-08,4.159042e-08,1.298889e-10,2.477991e-10,3.977665e-08,1.492402e-07,4.807832e-09,4.905053e-09
4,0.000068,2.479880e-07,3.598368e-09,1.182946e-07,9.985502e-01,4.501874e-08,3.008628e-08,7.166903e-10,5.058537e-05,7.454681e-07,...,1.901996e-08,9.505955e-10,1.540018e-08,2.381099e-06,1.605256e-09,4.692861e-10,1.090001e-07,3.523083e-07,4.888494e-08,8.795493e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.000006,2.134346e-08,1.901547e-08,1.343224e-09,2.480220e-09,6.748727e-09,2.446821e-07,2.233109e-04,3.043083e-08,1.936145e-04,...,1.140123e-07,5.733085e-09,9.295870e-08,7.790086e-07,1.076165e-06,9.986933e-01,9.390228e-09,7.668633e-07,1.256641e-08,1.911967e-08
205,0.000274,9.991363e-07,2.705714e-09,4.430732e-08,1.158293e-07,2.722195e-07,2.979251e-08,2.672837e-10,5.838190e-07,3.006369e-06,...,7.043756e-09,3.535571e-10,5.731386e-09,2.425590e-07,6.473211e-09,1.888524e-09,9.986956e-01,1.422702e-06,2.583798e-07,3.227646e-07
206,0.000130,4.749663e-07,4.995475e-08,2.590388e-08,5.508501e-08,2.911215e-05,6.241729e-06,2.873543e-09,1.176280e-06,3.614459e-05,...,7.883426e-08,3.800307e-09,6.160320e-08,2.048398e-07,7.782513e-08,2.270499e-08,2.092886e-07,9.987901e-01,5.429448e-05,8.265087e-05
207,0.000134,4.879235e-07,1.182848e-08,1.035407e-08,5.655116e-08,7.261605e-06,7.561315e-07,3.485812e-10,2.362261e-07,4.380406e-06,...,9.585999e-09,5.710169e-10,7.473406e-09,1.185414e-07,9.431717e-09,2.751718e-09,4.335259e-07,4.015449e-04,9.986685e-01,2.733031e-04


## Write to .mg format for VGsim


In [48]:
base_file = './output/manypop'

In [49]:
with open(base_file+'.mg', 'w') as f:
     f.write('#Migration_format_version 0.0.1\n')

probability_matrix.to_csv(base_file+'.mg', header=False, index=None, mode="a")
np.savetxt(base_file+'_country_codes.csv', population_sizes.index, delimiter=",", fmt='% s')


In [52]:
population_data = pd.DataFrame({'id': range(len(population_sizes['population'])),
                                'size': population_sizes['population'],
                                'contactDensity': np.full(shape=len(population_sizes['population']), fill_value=1.00,
                                                          dtype=np.double),
                                'conDenAfterLD': np.full(shape=len(population_sizes['population']), fill_value=0.1,
                                                         dtype=np.double),
                                'startLD': np.full(shape=len(population_sizes['population']), fill_value=0.01,
                                                   dtype=np.double),
                                'endLD': np.full(shape=len(population_sizes['population']), fill_value=0.002,
                                                 dtype=np.double),
                                'samplingMultiplier': np.full(shape=len(population_sizes['population']),
                                                              fill_value=1.00,
                                                              dtype=np.double)})
with open(base_file+'.pp', 'w') as f:
     f.write('#Population_format_version 0.0.1\n')

population_data.to_csv(base_file+'.pp', header=True, index=None, mode="a", sep=' ')
